# Nearest Neighbors

When exploring a large set of documents -- such as Wikipedia, news articles, StackOverflow, etc. -- it can be useful to get a list of related material. To find relevant documents you typically
* Decide on a notion of similarity
* Find the documents that are most similar 

In the assignment you will
* Gain intuition for different notions of similarity and practice finding similar documents. 
* Explore the tradeoffs with representing documents using raw word counts and TF-IDF
* Explore the behavior of different distance metrics by looking at the Wikipedia pages most similar to President Obama’s page.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import necessary packages

As usual we need to first import the Python packages that we will need.

The following code block will check if you have the correct version of GraphLab Create. Any version later than 1.8.5 will do. To upgrade, read [this page](https://turi.com/download/upgrade-graphlab-create.html).

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.sparse import csr_matrix      # sparse matrices



## Load Wikipedia dataset

We will be using the same dataset of Wikipedia pages that we used in the Machine Learning Foundations course (Course 1). Each element of the dataset consists of a link to the wikipedia article, the name of the person, and the text of the article (in lowercase).  

In [5]:
wiki = pd.read_csv('people_wiki.csv')
wiki = pd.DataFrame(wiki)
wiki['id'] = range(0, len(wiki))


In [6]:
wiki.loc[0]

URI           <http://dbpedia.org/resource/Digby_Morrell>
name                                        Digby Morrell
text    digby morrell born 10 october 1979 is a former...
id                                                      0
Name: 0, dtype: object

## Extract word count vectors

As we have seen in Course 1, we can extract word count vectors using a GraphLab utility function.  We add this as a column in `wiki`.

In [7]:
#wiki['word_count'] = graphlab.text_analytics.count_words(wiki['text'])
#wiki['word_count'] = wiki['text'].apply(lambda x: len(str(x).split(' ')))
import re
def remove_punctuation(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

In [8]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

In [9]:
import json
from pprint import pprint

with open('people_wiki_map_index_to_word.json') as data_file:    
    map_index_to_word = json.load(data_file)


In [10]:
word_count = load_sparse_csr('people_wiki_word_count.npz')

In [11]:
print word_count.shape

(59071, 547979)


In [12]:
wiki.head()

,URI,name,text,id
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...,0
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...,1
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...,2
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...,3
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...,4


In [13]:
#temp =  wiki.iloc[0:1]['text']
#temp1 = Counter(pd.Series(temp))
#print temp1


In [14]:
def unpack_dict(matrix, map_index_to_word):
   
    # if you're not using SFrame, replace this line with
    table = sorted(map_index_to_word, key=map_index_to_word.get)
    
    
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    
    num_doc = matrix.shape[0]

    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} \
               for i in xrange(num_doc) ]

wiki['word_count'] = unpack_dict(word_count, map_index_to_word)

In [15]:
print(wiki.shape)

(59071, 5)


r## Find nearest neighbors

In [16]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='euclidean', algorithm='brute')
model.fit(word_count)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

Let's start by finding the nearest neighbors of the Barack Obama page using the word count vectors to represent the articles and Euclidean distance to measure distance.  For this, again will we use a GraphLab Create implementation of nearest neighbor search.

In [17]:
print wiki[wiki['name'] == 'Barack Obama']['id']


35817    35817
Name: id, dtype: int64


Let's look at the top 10 nearest neighbors by performing the following query:

In [18]:
#index of obama = 35817
distances, indices = model.kneighbors(word_count[35817], n_neighbors=10) # 1st arg: word count vector
neighbors = pd.DataFrame({'distance':distances.flatten(), 'id':indices.flatten()})
#print wiki.join(neighbors, on='id').sort('distance')[['id','name','distance']]
print neighbors
print wiki.shape
print indices[0]
wiki_obama = wiki.loc[indices[0]]
wiki_obama = wiki_obama.loc[:,['id','name']]
print pd.merge(wiki_obama, neighbors, on='id')
#print wiki_obama.join(neighbors,on = 'id').sort('distances')


    distance     id
0   0.000000  35817
1  33.075671  24478
2  34.394767  28447
3  36.152455  35357
4  36.166283  14754
5  36.331804  13229
6  36.400549  31423
7  36.496575  22745
8  36.633318  36364
9  36.959437   9210
(59071, 5)
[35817 24478 28447 35357 14754 13229 31423 22745 36364  9210]
      id                        name   distance
0  35817                Barack Obama   0.000000
1  24478                   Joe Biden  33.075671
2  28447              George W. Bush  34.394767
3  35357            Lawrence Summers  36.152455
4  14754                 Mitt Romney  36.166283
5  13229            Francisco Barrio  36.331804
6  31423              Walter Mondale  36.400549
7  22745  Wynn Normington Hugh-Jones  36.496575
8  36364                  Don Bonker  36.633318
9   9210                Andy Anstett  36.959437


In [19]:

tmp = wiki

All of the 10 people are politicians, but about half of them have rather tenuous connections with Obama, other than the fact that they are politicians.

* Francisco Barrio is a Mexican politician, and a former governor of Chihuahua.
* Walter Mondale and Don Bonker are Democrats who made their career in late 1970s.
* Wynn Normington Hugh-Jones is a former British diplomat and Liberal Party official.
* Andy Anstett is a former politician in Manitoba, Canada.

Nearest neighbors with raw word counts got some things right, showing all politicians in the query result, but missed finer and important details.

For instance, let's find out why Francisco Barrio was considered a close neighbor of Obama.  To do this, let's look at the most frequently used words in each of Barack Obama and Francisco Barrio's pages:

In [20]:
def get_id(name,data):
    name_id = data[data.name == name]['id'].values[0]
    #name_id = name_id[0]
    return name_id
def top_words(name_id):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    #name_id = wiki[wiki.name ==name]['id']
    row = wiki.iloc[name_id]
    print row
    d = row['word_count']
    word_count_table = pd.DataFrame(d.items(),columns=['word','count'])
    #print(word_count_table.sort('count',ascending=False))

    return word_count_table.sort('count', ascending=False)


In [21]:
name_id = get_id('Barack Obama',tmp)
print name_id
obama_words = top_words(name_id)
obama_words
name_id = get_id('Francisco Barrio',tmp)
barrio_words = top_words(name_id)
barrio_words

35817
URI                  <http://dbpedia.org/resource/Barack_Obama>
name                                               Barack Obama
text          barack hussein obama ii brk husen bm born augu...
id                                                        35817
word_count    {u'operations': 1, u'represent': 1, u'office':...
Name: 35817, dtype: object
URI              <http://dbpedia.org/resource/Francisco_Barrio>
name                                           Francisco Barrio
text          francisco javier barrio terrazas born november...
id                                                        13229
word_count    {u'sector': 1, u'ambassador': 1, u'mexican': 2...
Name: 13229, dtype: object


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,word,count
220,the,36
84,of,24
206,and,18
184,in,17
125,he,10
17,to,9
14,chihuahua,7
159,governor,6
211,a,6
181,as,5


In [22]:
barrio_words.rename(columns = {barrio_words.columns[1]:'barrioCount'},inplace=True)
print barrio_words.head(10)

          word  barrioCount
220        the           36
84          of           24
206        and           18
184         in           17
125         he           10
17          to            9
14   chihuahua            7
159   governor            6
211          a            6
181         as            5


Let's extract the list of most frequent words that appear in both Obama's and Barrio's documents. We've so far sorted all words from Obama and Barrio's articles by their word frequencies. We will now use a dataframe operation known as **join**. The **join** operation is very useful when it comes to playing around with data: it lets you combine the content of two tables using a shared column (in this case, the word column). See [the documentation](https://dato.com/products/create/docs/generated/graphlab.SFrame.join.html) for more details.

For instance, running
```
obama_words.join(barrio_words, on='word')
```
will extract the rows from both tables that correspond to the common words.

In [23]:
#combined_words = obama_words.join(barrio_words, on='word')
combined_words = obama_words.join(barrio_words.set_index('word'), on='word')


Since both tables contained the column named `count`, SFrame automatically renamed one of them to prevent confusion. Let's rename the columns to tell which one is for which. By inspection, we see that the first column (`count`) is for Obama and the second (`count.1`) for Barrio.

In [24]:
combined_words.sort('count', ascending=False).head(10)

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,word,count,barrioCount
23,the,40,36.0
215,in,30,17.0
138,and,21,18.0
127,of,18,24.0
161,to,14,9.0
37,his,11,5.0
189,obama,9,NaN
133,act,8,NaN
260,a,7,6.0
156,he,7,10.0


**Quiz Question**. Among the words that appear in both Barack Obama and Francisco Barrio, take the 5 that appear most frequently in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

Hint:
* Refer to the previous paragraph for finding the words that appear in both articles. Sort the common words by their frequencies in Obama's article and take the largest five.
* Each word count vector is a Python dictionary. For each word count vector in SFrame, you'd have to check if the set of the 5 common words is a subset of the keys of the word count vector. Complete the function `has_top_words` to accomplish the task.
  - Convert the list of top 5 words into set using the syntax
```
set(common_words)
```
    where `common_words` is a Python list. See [this link](https://docs.python.org/2/library/stdtypes.html#set) if you're curious about Python sets.
  - Extract the list of keys of the word count dictionary by calling the [`keys()` method](https://docs.python.org/2/library/stdtypes.html#dict.keys).
  - Convert the list of keys into a set as well.
  - Use [`issubset()` method](https://docs.python.org/2/library/stdtypes.html#set) to check if all 5 words are among the keys.
* Now apply the `has_top_words` function on every row of the SFrame.
* Compute the sum of the result column to obtain the number of articles containing all the 5 top words.

In [25]:
common_words = combined_words.iloc[0:5]['word']

def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = set(word_count_vector.keys())   # YOUR CODE HERE
    
    # return True if common_words is a subset of unique_words
    # return False otherwise
    testSet = set(common_words)
    #print testSet
    yes = testSet.issubset(unique_words)
    return yes  # YOUR CODE HERE


wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)
wiki['has_top_words'].sum() # YOUR CODE HERE


56066

**Checkpoint**. Check your `has_top_words` function on two random articles:

In [26]:
print 'Output from your function:', has_top_words(wiki.iloc[32]['word_count'])
print 'Correct output: True'
print 'Also check the length of unique_words. It should be 167'

Output from your function: True
Correct output: True
Also check the length of unique_words. It should be 167


In [27]:
print 'Output from your function:', has_top_words(wiki.iloc[33]['word_count'])
print 'Correct output: False'
print 'Also check the length of unique_words. It should be 188'

Output from your function: False
Correct output: False
Also check the length of unique_words. It should be 188


In [28]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics.pairwise import euclidean_distances

v = DictVectorizer(sparse=False)
D = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}]
def dist(name1_dict,name2_dict,name3_dict):
    D =[name1_dict,name2_dict,name3_dict]
    X = v.fit_transform(D)
    distance = euclidean_distances(X,X)
    return distance

**Quiz Question**. Measure the pairwise distance between the Wikipedia pages of Barack Obama, George W. Bush, and Joe Biden. Which of the three pairs has the smallest distance?

Hint: To compute the Euclidean distance between two dictionaries, use `graphlab.toolkits.distances.euclidean`. Refer to [this link](https://dato.com/products/create/docs/generated/graphlab.toolkits.distances.euclidean.html) for usage.

In [29]:
obama_id = get_id('Barack Obama',tmp)
bush_id = get_id('George W. Bush',tmp)
joe_id = get_id('Joe Biden',tmp)
obama_dict = wiki.iloc[obama_id]['word_count']
bush_dict = wiki.iloc[bush_id]['word_count']
joe_dict = wiki.iloc[joe_id]['word_count']

distance = dist(obama_dict,bush_dict,joe_dict)
print distance

[[  0.          34.39476704  33.07567082]
 [ 34.39476704   0.          32.75667871]
 [ 33.07567082  32.75667871   0.        ]]


**Quiz Question**. Collect all words that appear both in Barack Obama and George W. Bush pages.  Out of those words, find the 10 words that show up most often in Obama's page. 

In [30]:
obama_words = set(obama_dict.keys())
bush_words = set(bush_dict.keys())
common_words_obama_bush =list(obama_words.intersection(bush_words))
print len(common_words_obama_bush)
top_words_obama = top_words(obama_id)
topcommon = top_words_obama[top_words_obama.word.isin(common_words_obama_bush)]
print topcommon.head(10)

86
URI                     <http://dbpedia.org/resource/Barack_Obama>
name                                                  Barack Obama
text             barack hussein obama ii brk husen bm born augu...
id                                                           35817
word_count       {u'operations': 1, u'represent': 1, u'office':...
has_top_words                                                 True
Name: 35817, dtype: object
    word  count
23   the     40
215   in     30
138  and     21
127   of     18
161   to     14
37   his     11
133  act      8
156   he      7
260    a      7
221   as      6


C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


**Note.** Even though common words are swamping out important subtle differences, commonalities in rarer political words still matter on the margin. This is why politicians are being listed in the query result instead of musicians, for example. In the next subsection, we will introduce a different metric that will place greater emphasis on those rarer words.

## TF-IDF to the rescue

Much of the perceived commonalities between Obama and Barrio were due to occurrences of extremely frequent words, such as "the", "and", and "his". So nearest neighbors is recommending plausible results sometimes for the wrong reasons. 

To retrieve articles that are more relevant, we should focus more on rare words that don't happen in every article. **TF-IDF** (term frequency–inverse document frequency) is a feature representation that penalizes words that are too common.  Let's use GraphLab Create's implementation of TF-IDF and repeat the search for the 10 nearest neighbors of Barack Obama:

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
v.fit(wiki['text'])
vector = v.transform(wiki['text'])



In [32]:
model_tf_idf = NearestNeighbors(metric='euclidean', algorithm='brute')
model_tf_idf.fit(vector)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='euclidean',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [33]:
distances1, indices1 = model_tf_idf.kneighbors(vector[35817], n_neighbors=10) # 1st arg: word count vector
neighbors1 = pd.DataFrame({'distance':distances1.flatten(), 'id':indices1.flatten()})
print neighbors1
print wiki.shape
print indices1[0]
wiki_obama = wiki.loc[indices1[0]]
wiki_obama = wiki_obama.loc[:,['id','name']]
print pd.merge(wiki_obama, neighbors1, on='id')

   distance     id
0  0.000000  35817
1  1.068439  24478
2  1.109896  57108
3  1.118028  38376
4  1.139969  38714
5  1.147769  28447
6  1.150374  39357
7  1.154939  48693
8  1.157761  18827
9  1.159678  46811
(59071, 6)
[35817 24478 57108 38376 38714 28447 39357 48693 18827 46811]
      id                     name  distance
0  35817             Barack Obama  0.000000
1  24478                Joe Biden  1.068439
2  57108   Hillary Rodham Clinton  1.109896
3  38376           Samantha Power  1.118028
4  38714  Eric Stern (politician)  1.139969
5  28447           George W. Bush  1.147769
6  39357              John McCain  1.150374
7  48693              Artur Davis  1.154939
8  18827             Henry Waxman  1.157761
9  46811            Jeff Sessions  1.159678


Let's determine whether this list makes sense.
* With a notable exception of Roland Grossenbacher, the other 8 are all American politicians who are contemporaries of Barack Obama.
* Phil Schiliro, Jesse Lee, Samantha Power, and Eric Stern worked for Obama.

Clearly, the results are more plausible with the use of TF-IDF. Let's take a look at the word vector for Obama and Schilirio's pages. Notice that TF-IDF representation assigns a weight to each word. This weight captures relative importance of that word in the document. Let us sort the words in Obama's article by their TF-IDF weights; we do the same for Schiliro's article as well.

In [40]:
# get the weight of word

word_feature = v.get_feature_names()

def get_word_weight(name_id):
    text_of_this_id = wiki.iloc[name_id]['text']
    text_dict = dict.fromkeys(word_feature,0)
    # get the value matrix 
    values  = vector[name_id]
    weight = [values[0,i] for i in range(0,len(word_feature))]
    text_dict = dict(zip(word_feature,weight))
    #removw value 0
    text_dict = {key:value for (key,value) in text_dict.iteritems() if value !=0}
    return text_dict

def get_id(name):
    name_id = wiki[wiki.name == name]['id'].values[0]
    #name_id = name_id[0]
    return name_id

def get_tfidf(name):
    name_id = get_id(name)
    text_dict = get_word_weight(name_id)
    
    return text_dict

In [41]:
test = get_tfidf('Barack Obama')

In [ ]:
wiki['tf_idf'] = wiki['name'].apply(get_tfidf)

In [ ]:

def top_words_tf_idf(name):
    """
    Get a table of the most frequent words in the given person's wikipedia page.
    """
    name_id = wiki[wiki.name ==name]['id'].values[0]
    row = wiki.iloc[name_id]
    d = row['tf_idf']
    print d
    word_count_table = pd.DataFrame(d.items(),columns=['word','weight'])
    #print(word_count_table.sort('count',ascending=False))

    return word_count_table.sort('weight', ascending=False)
    

In [ ]:
dict1 = get_word_weight(0)
#print dict1
wiki['word_weight']

In [ ]:

obama_tf_idf = top_words_tf_idf('Barack Obama')
obama_tf_idf

In [ ]:
schiliro_tf_idf = top_words_tf_idf('Phil Schiliro')
schiliro_tf_idf

Using the **join** operation we learned earlier, try your hands at computing the common words shared by Obama's and Schiliro's articles. Sort the common words by their TF-IDF weights in Obama's document.

The first 10 words should say: Obama, law, democratic, Senate, presidential, president, policy, states, office, 2011.

**Quiz Question**. Among the words that appear in both Barack Obama and Phil Schiliro, take the 5 that have largest weights in Obama. How many of the articles in the Wikipedia dataset contain all of those 5 words?

In [ ]:
common_words = ...  # YOUR CODE HERE

def has_top_words(word_count_vector):
    # extract the keys of word_count_vector and convert it to a set
    unique_words = ...   # YOUR CODE HERE
    # return True if common_words is a subset of unique_words
    # return False otherwise
    return ...  # YOUR CODE HERE

wiki['has_top_words'] = wiki['word_count'].apply(has_top_words)

# use has_top_words column to answer the quiz question
...  # YOUR CODE HERE

Notice the huge difference in this calculation using TF-IDF scores instead  of raw word counts. We've eliminated noise arising from extremely common words.

## Choosing metrics

You may wonder why Joe Biden, Obama's running mate in two presidential elections, is missing from the query results of `model_tf_idf`. Let's find out why. First, compute the distance between TF-IDF features of Obama and Biden.

**Quiz Question**. Compute the Euclidean distance between TF-IDF features of Obama and Biden. Hint: When using Boolean filter in SFrame/SArray, take the index 0 to access the first match.

The distance is larger than the distances we found for the 10 nearest neighbors, which we repeat here for readability:

In [ ]:
model_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=10)

But one may wonder, is Biden's article that different from Obama's, more so than, say, Schiliro's? It turns out that, when we compute nearest neighbors using the Euclidean distances, we unwittingly favor short articles over long ones. Let us compute the length of each Wikipedia document, and examine the document lengths for the 100 nearest neighbors to Obama's page.

In [ ]:
def compute_length(row):
    return len(row['text'].split(' '))

wiki['length'] = wiki.apply(compute_length) 

In [ ]:
nearest_neighbors_euclidean = model_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
nearest_neighbors_euclidean = nearest_neighbors_euclidean.join(wiki[['name', 'length']], on={'reference_label':'name'})

In [ ]:
nearest_neighbors_euclidean.sort('rank')

To see how these document lengths compare to the lengths of other documents in the corpus, let's make a histogram of the document lengths of Obama's 100 nearest neighbors and compare to a histogram of document lengths for all documents.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])

plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size':16})
plt.tight_layout()

Relative to the rest of Wikipedia, nearest neighbors of Obama are overwhemingly short, most of them being shorter than 300 words. The bias towards short articles is not appropriate in this application as there is really no reason to  favor short articles over long articles (they are all Wikipedia articles, after all). Many of the Wikipedia articles are 300 words or more, and both Obama and Biden are over 300 words long.

**Note**: For the interest of computation time, the dataset given here contains _excerpts_ of the articles rather than full text. For instance, the actual Wikipedia article about Obama is around 25000 words. Do not be surprised by the low numbers shown in the histogram.

**Note:** Both word-count features and TF-IDF are proportional to word frequencies. While TF-IDF penalizes very common words, longer articles tend to have longer TF-IDF vectors simply because they have more words in them.

To remove this bias, we turn to **cosine distances**:
$$
d(\mathbf{x},\mathbf{y}) = 1 - \frac{\mathbf{x}^T\mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|}
$$
Cosine distances let us compare word distributions of two articles of varying lengths.

Let us train a new nearest neighbor model, this time with cosine distances.  We then repeat the search for Obama's 100 nearest neighbors.

In [ ]:
model2_tf_idf = graphlab.nearest_neighbors.create(wiki, label='name', features=['tf_idf'],
                                                  method='brute_force', distance='cosine')

In [ ]:
nearest_neighbors_cosine = model2_tf_idf.query(wiki[wiki['name'] == 'Barack Obama'], label='name', k=100)
nearest_neighbors_cosine = nearest_neighbors_cosine.join(wiki[['name', 'length']], on={'reference_label':'name'})

In [ ]:
nearest_neighbors_cosine.sort('rank')

From a glance at the above table, things look better.  For example, we now see Joe Biden as Barack Obama's nearest neighbor!  We also see Hillary Clinton on the list.  This list looks even more plausible as nearest neighbors of Barack Obama.

Let's make a plot to better visualize the effect of having used cosine distance in place of Euclidean on our TF-IDF vectors.

In [ ]:
plt.figure(figsize=(10.5,4.5))
plt.figure(figsize=(10.5,4.5))
plt.hist(wiki['length'], 50, color='k', edgecolor='None', histtype='stepfilled', normed=True,
         label='Entire Wikipedia', zorder=3, alpha=0.8)
plt.hist(nearest_neighbors_euclidean['length'], 50, color='r', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (Euclidean)', zorder=10, alpha=0.8)
plt.hist(nearest_neighbors_cosine['length'], 50, color='b', edgecolor='None', histtype='stepfilled', normed=True,
         label='100 NNs of Obama (cosine)', zorder=11, alpha=0.8)
plt.axvline(x=wiki['length'][wiki['name'] == 'Barack Obama'][0], color='k', linestyle='--', linewidth=4,
           label='Length of Barack Obama', zorder=2)
plt.axvline(x=wiki['length'][wiki['name'] == 'Joe Biden'][0], color='g', linestyle='--', linewidth=4,
           label='Length of Joe Biden', zorder=1)
plt.axis([0, 1000, 0, 0.04])
plt.legend(loc='best', prop={'size':15})
plt.title('Distribution of document length')
plt.xlabel('# of words')
plt.ylabel('Percentage')
plt.rcParams.update({'font.size': 16})
plt.tight_layout()

Indeed, the 100 nearest neighbors using cosine distance provide a sampling across the range of document lengths, rather than just short articles like Euclidean distance provided.

**Moral of the story**: In deciding the features and distance measures, check if they produce results that make sense for your particular application.

# Problem with cosine distances: tweets vs. long articles

Happily ever after? Not so fast. Cosine distances ignore all document lengths, which may be great in certain situations but not in others. For instance, consider the following (admittedly contrived) example.

```
+--------------------------------------------------------+
|                                             +--------+ |
|  One that shall not be named                | Follow | |
|  @username                                  +--------+ |
|                                                        |
|  Democratic governments control law in response to     |
|  popular act.                                          |
|                                                        |
|  8:05 AM - 16 May 2016                                 |
|                                                        |
|  Reply   Retweet (1,332)   Like (300)                  |
|                                                        |
+--------------------------------------------------------+
```

How similar is this tweet to Barack Obama's Wikipedia article? Let's transform the tweet into TF-IDF features, using an encoder fit to the Wikipedia dataset.  (That is, let's treat this tweet as an article in our Wikipedia dataset and see what happens.)

In [ ]:
sf = graphlab.SFrame({'text': ['democratic governments control law in response to popular act']})
sf['word_count'] = graphlab.text_analytics.count_words(sf['text'])

encoder = graphlab.feature_engineering.TFIDF(features=['word_count'], output_column_prefix='tf_idf')
encoder.fit(wiki)
sf = encoder.transform(sf)
sf

Let's look at the TF-IDF vectors for this tweet and for Barack Obama's Wikipedia entry, just to visually see their differences.

In [ ]:
tweet_tf_idf = sf[0]['tf_idf.word_count']
tweet_tf_idf

In [ ]:
obama = wiki[wiki['name'] == 'Barack Obama']
obama

Now, compute the cosine distance between the Barack Obama article and this tweet:

In [ ]:
obama_tf_idf = obama[0]['tf_idf']
graphlab.toolkits.distances.cosine(obama_tf_idf, tweet_tf_idf)

Let's compare this distance to the distance between the Barack Obama article and all of its Wikipedia 10 nearest neighbors:

In [ ]:
model2_tf_idf.query(obama, label='name', k=10)

With cosine distances, the tweet is "nearer" to Barack Obama than everyone else, except for Joe Biden!  This probably is not something we want. If someone is reading the Barack Obama Wikipedia page, would you want to recommend they read this tweet? Ignoring article lengths completely resulted in nonsensical results. In practice, it is common to enforce maximum or minimum document lengths. After all, when someone is reading a long article from _The Atlantic_, you wouldn't recommend him/her a tweet.